In [1]:
from dotenv import load_dotenv
import openai
import os
from pathlib import Path  # Python 3.6+ only

env_path = Path('../') / 'secrets.env'
load_dotenv(dotenv_path=env_path)
key = os.getenv("AZURE_OPENAI_API_KEY")
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_type = "azure"
openai.api_version = "2023-05-15" 
openai.api_key= key
openai.api_base=openai_endpoint
engine = "gpt-35-turbo"

In [12]:
messages = [{"role":"system","content":"You are an AI assistant that helps people find information."},
            {"role":"user","content":"extract following content into json format \n<<content>>\n\nHello and welcome to our customer support. My name is [Your Name] and I am here to assist you. How may I help you today?\n\nCustomer: Hi, I am having trouble with my account.\n\nYou: I am sorry to hear that. Can you please provide me with your account information so that I can assist you better?\n\nCustomer: Sure, my account username is [Username].\n\nYou: Thank you for providing that information. Can you please tell me what specific problem you are facing with your account?\n\nCustomer: I am unable to access my account. It keeps showing an error message.\n\nYou: I apologize for the inconvenience. Let me check your account and see what the problem is. Can you please provide me with your registered email address?\n\nCustomer: My email address is [Email Address].\n\nYou: Thank you. Let me check your account. [Check account]. I see that there is an issue with your password. Can you please try resetting your password by clicking on the \"forgot password\" link on the login page and following the steps?\n\nCustomer: Okay, I will try that.\n\nYou: Great. Once you have reset your password, please try logging in again and let me know if the issue persists.\n\nCustomer: Okay, I will do that.\n\nYou: Is there anything else I can assist you with?\n\nCustomer: No, that's all. Thank you for your help.\n\nYou: You're welcome. If you have any further questions or concerns, please feel free to contact us again. Have a great day!\n\n\nThe extracted content is \n{\"summary\": \"summary place holder\", \"problem\":[\"problem place holder\",\"problem place holder\"], \"next_action\":[\"next action place holder\",\"next action place holder\"]}"},
            ]


In [2]:
messages = [
    {"role": "system", "content": """You are an assistant designed to extract entities from text. Users will paste in a string of text and you will respond with entities you've extracted from the text as a JSON object. Here's an example of your output format:
{
   "name": "",
   "company": "",
   "phone_number": ""
}"""},
{"role": "user", "content": "Hello. My name is Robert Smith. I'm calling from Contoso Insurance, Delaware. My colleague mentioned that you are interested in learning about our comprehensive benefits policy. Could you give me a call back at (555) 346-9322 when you get a chance so we can go over the benefits?"}
]

In [3]:
response = openai.ChatCompletion.create(
engine=engine, 
messages = messages)
llm_output = response['choices'][0]['message']['content']
llm_output

'Based on the text you provided, here are the entities that I have extracted:\n\n {\n   "name": "Robert Smith",\n   "company": "Contoso Insurance",\n   "phone_number": "(555) 346-9322"\n}'

In [15]:
import json
json.loads(llm_output)

{'summary': 'The customer is experiencing trouble accessing their account and is receiving an error message. It is discovered that the issue is with their password and they are instructed to reset it. The customer confirms they will do so and no further assistance is needed.',
 'problem': ['Customer is unable to access their account and is seeing an error message.'],
 'next_action': ['Ask for account information, including username and email.',
  'Check the account and discover that the issue is with the password.',
  'Instruct the customer to reset their password and log in again.',
  'Confirm with the customer that no further assistance is needed.']}

In [8]:
from azure.storage.blob import BlobServiceClient
con_str ="DefaultEndpointsProtocol=https;AccountName=adlsstore0001;AccountKey=fxjZJXVwuMUS6mTu2JhfmcnLZ68RY1anPYV1o2jWJce5MS4RttZIYzrXaKjJ39Nes7LaxeMiIbgE+ASt8QTLqQ==;EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(con_str)

container_client = blob_service_client.get_container_client("mltraining")


In [34]:
from azure.storage.blob import generate_container_sas, generate_account_sas

sas_token = generate_account_sas(
    container_client.account_name,
    permission= "p",
    account_key=container_client.credential.account_key,
    resource_types="p",
    expiry
)
sas_token

TypeError: generate_account_sas() missing 1 required positional argument: 'expiry'

In [17]:
import os, uuid, sys
from azure.storage.filedatalake import DataLakeServiceClient
from azure.core._match_conditions import MatchConditions
from azure.storage.filedatalake._models import ContentSettings
from azure.identity import DefaultAzureCredential

In [23]:
service_client  = DataLakeServiceClient(sas_token=sas_token,account_url="{}://{}.dfs.core.windows.net".format(
            "https", "adlsstore0001"))
file_system_client = service_client.get_file_system_client(file_system="mltraining")

directory_client = file_system_client.get_directory_client("dev")

acl_props = directory_client.get_access_control()



print(acl_props['permissions'])

new_dir_permissions = "rwxr-xrw-"

directory_client.set_access_control(permissions=new_dir_permissions)

acl_props = directory_client.get_access_control()

print(acl_props['permissions'])
    


ClientAuthenticationError: Operation returned an invalid status 'Server failed to authenticate the request. Please refer to the information in the www-authenticate header.'
ErrorCode:NoAuthenticationInformation

In [32]:
con_str2 =f"BlobEndpoint=https://adlsstore0001.blob.core.windows.net/;SharedAccessSignature=dsdf"
service_client=DataLakeServiceClient.from_connection_string(con_str2)
file_system_client = service_client.get_file_system_client(file_system="mltraining")

directory_client = file_system_client.get_directory_client("dev")

acl_props = directory_client.get_access_control()



print(acl_props['permissions'])

new_dir_permissions = "rwxr-xrw-"

directory_client.set_access_control(permissions=new_dir_permissions)

acl_props = directory_client.get_access_control()

print(acl_props['permissions'])
    


ClientAuthenticationError: Operation returned an invalid status 'Server failed to authenticate the request. Make sure the value of Authorization header is formed correctly including the signature.'
ErrorCode:AuthenticationFailed